In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [7]:
#! pip install pyarrow

In [8]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet', engine='pyarrow')

In [10]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet', engine='pyarrow')

In [11]:
df_jan.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0


In [12]:
df_feb.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.0,4.39,1.0,N,68,236,1,20.5,1.0,0.5,1.28,0.0,1.0,26.78,2.5,0.0
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.0,7.71,1.0,N,48,243,1,31.0,1.0,0.5,9.00,0.0,1.0,45.00,2.5,0.0


In [18]:
assert (df_jan.columns == df_feb.columns).all(), "Not same schema"

In [19]:
df_jan.shape

(2964624, 19)

In [20]:
df_feb.shape

(3007526, 19)

### Q1 Read the data for January. How many columns are there?

#### Answer Q1 : 19  colums

### Q2 Computing duration : What's the standard deviation of the trips duration in January?



In [56]:
# duration in minutes : dropoff - pickup
df_jan["duration"] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).dt.total_seconds() // 60

In [57]:
df_jan.duration

0          19.0
1           6.0
2          17.0
3           8.0
4           6.0
           ... 
2964619     8.0
2964620    14.0
2964621    19.0
2964622    17.0
2964623    15.0
Name: duration, Length: 2964624, dtype: float64

In [74]:
df_jan.duration.describe()

count    2.964624e+06
mean     1.512550e+01
std      3.485121e+01
min     -1.400000e+01
25%      7.000000e+00
50%      1.100000e+01
75%      1.800000e+01
max      9.455000e+03
Name: duration, dtype: float64

#### Answer  Q2: Around 34 so the closest answer is 32.59 

### Q3. Dropping outliers : What fraction of the records left after you dropped the outliers?

In [75]:
import numpy as np 
df_jan["outlier"] = np.logical_and(df_jan.duration >=1, df_jan.duration <=60)


In [77]:
df_jan.outlier.value_counts(normalize=True)

True     0.978554
False    0.021446
Name: outlier, dtype: float64